In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
HOME = expanduser("~")
import os, sys
import SimpleITK as sitk
#from PIL import Image, ImageOps
#from scipy.ndimage import rotate
from tqdm import tqdm
#from IPython.display import clear_output
#REPO_DIR = '/home/eddyod/programming/pipeline_utility'
#sys.path.append(REPO_DIR)
#from utilities.sqlcontroller import SqlController
#from utilities.file_location import FileLocationManager
#stack = 'DK39'
#sqlController = SqlController()
#fileLocationManager = FileLocationManager(stack)
#image_name_list = sqlController.get_image_list(stack, 'destination')
#%matplotlib inline

In [4]:
#DIR = os.path.join(HOME, 'programming', 'dk39')
DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39'
ORIENTED = os.path.join(DIR, 'preps', 'resized')
PREALIGNED = os.path.join(DIR, 'preps', 'prealigned')
ALIGNED = os.path.join(DIR, 'preps', 'aligned')
NORMALIZED = os.path.join(DIR, 'preps', 'normalized')
POSTALIGNED = os.path.join(DIR, 'preps', 'postaligned')
PADDED = os.path.join(DIR, 'preps', 'padded')
RESIZED = os.path.join(DIR, 'preps', 'resized')
INPUT = RESIZED
OUTPUT = ALIGNED
#BADS = ['DK39_ID_0001_slide001_S1_C1.tif', 'DK39_ID_0007_slide001_S2_C1.tif', 'DK39_ID_0010_slide001_S3_C1.tif']
#INPUTS = INPUTS[80:120]
#COLOR = 2**16
files = sorted(os.listdir(INPUT))

In [6]:
def simple_resample(fixed_image, moving_image):
    #parameterMap = sitk.GetDefaultParameterMap('affine')
    #elastixImageFilter = sitk.ElastixImageFilter()
    #elastixImageFilter.SetParameterMap(parameterMap)
    #elastixImageFilter.Execute()
    #return  sitk.ElastixImageFilter(fixed_image, moving_image)
    
    #parameterMap = sitk.GetDefaultParameterMap("rigid")
    #parameterMap["Transform"] = ["AffineTransform"]

    #elastixImageFilter = sitk.ElastixImageFilter()
    #elastixImageFilter.SetFixedImage(fixed_image)
    #elastixImageFilter.SetMovingImage(moving_image)
    #elastixImageFilter.SetParameterMap(parameterMap)
    #elastixImageFilter.Execute()
    #return elastixImageFilter.GetResultImage()
    return  sitk.Elastix(fixed_image, moving_image)



In [ ]:
movingFile = files[0]
movingPath = os.path.join(INPUT, movingFile)
fixedPath = movingPath
fixed_image =  sitk.ReadImage(fixedPath, sitk.sitkFloat32)
moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
fixed_image = simple_resample(fixed_image, moving_image)


for file in tqdm(files):
    movingFile = file
    movingPath = os.path.join(INPUT, movingFile)
    moving_image = sitk.ReadImage(movingPath, sitk.sitkFloat32) 
    fixed_image = simple_resample(fixed_image, moving_image)
    
    outfile = os.path.join(OUTPUT, file)
    flat = img.flatten()
    fmax = int(flat.max())
    fmin = int(flat.min())
    flat = flat + abs(fmin)
    img = np.reshape(flat, img.shape)    
    img[img <= 0] = 0
    io.imsave(outfile, img.astype('uint16'), check_contrast=False)
    img = None
print('done')

In [ ]:
start, finish = 0,10
for name,file in zip(file_inputs[start:finish], img_outputs[start:finish]):
    plt.style.use('classic')
    plt.figure()
    plt.title('{} {}'.format(name, file.shape))
    plt.imshow(file, cmap='gray')
    plt.show()